In [24]:
import matplotlib.pyplot as plt
import os
import pandas
from skimage import io, transform
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader

Step 1 : Load the data

In [10]:
class ImagesDataset(Dataset):
    """Images dataset."""

    def __init__(self, txt_file, root_dir, transform=None):
        """
        Args:
            txt_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.data = pandas.read_table(txt_file, delim_whitespace=True, names=('name', 'label'))
        print(self.data)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir,
                                self.data.iloc[idx, 0])
        image = io.imread(img_name)
        labels = self.data.iloc[idx, 1:].as_matrix()
        labels = labels.astype('float').reshape(-1, 2)
        sample = {'image': image, 'labels': labels}

        if self.transform:
            sample = self.transform(sample)

        return sample

In [18]:
not_face_dataset = ImagesDataset(txt_file='./start_deep/start_deep/negatives.txt',
                                    root_dir='./start_deep/start_deep/train_images/')

                             name  label
0       0/137021_102_88_72_72.pgm      0
1        0/137021_103_5_48_48.pgm      0
2       0/137021_115_30_60_60.pgm      0
3       0/137021_117_15_36_36.pgm      0
4       0/137021_117_38_57_57.pgm      0
5       0/137021_121_56_36_36.pgm      0
6      0/137021_123_131_36_36.pgm      0
7       0/137021_123_93_55_55.pgm      0
8         0/137021_12_4_75_75.pgm      0
9        0/137021_128_2_38_38.pgm      0
10     0/137021_135_119_45_45.pgm      0
11      0/137021_138_12_36_36.pgm      0
12      0/137021_138_99_36_36.pgm      0
13        0/137021_1_42_76_76.pgm      0
14       0/137021_14_68_42_42.pgm      0
15       0/137021_14_87_86_86.pgm      0
16       0/137021_14_94_55_55.pgm      0
17       0/137021_16_86_82_82.pgm      0
18       0/137021_17_40_73_73.pgm      0
19       0/137021_18_65_89_89.pgm      0
20        0/137021_19_8_36_36.pgm      0
21       0/137021_21_37_36_36.pgm      0
22       0/137021_22_43_66_66.pgm      0
23       0/13702

In [25]:
fig = plt.figure()

for i in range(len(not_face_dataset)):
    sample = not_face_dataset[i]

    print(i, sample['name'].shape, sample['label'].shape)

    ax = plt.subplot(1, 4, i + 1)
    plt.tight_layout()
    ax.set_title('Sample #{}'.format(i))
    ax.axis('off')
    show_landmarks(**sample)

    if i == 3:
        plt.show()
        break

ValueError: cannot reshape array of size 1 into shape (2)

In [35]:
imagenet_data = torchvision.datasets.ImageFolder('start_deep/start_deep/train_images/')
data_loader = torch.utils.data.DataLoader(imagenet_data, batch_size=4, shuffle=True, num_workers=2)

In [33]:
print(imagenet_data)

Dataset ImageFolder
    Number of datapoints: 91720
    Root Location: start_deep/start_deep/train_images/
    Transforms (if any): None
    Target Transforms (if any): None


In [36]:
print(data_loader)

Step 2 : Define neural network

In [38]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 6 * 6, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:] # all dimensions except the batc
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = Net()

Step 3 : Define a Loss function and optimizer

In [39]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

Step 4 : Train the network

In [ ]:
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')